In [11]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import sqlite3
import re
from datetime import datetime

In [12]:
# Connect to SQLite database
conn = sqlite3.connect("smartcity_complaints.db")
cursor = conn.cursor()

# Create table if not exists
cursor.execute('''
CREATE TABLE IF NOT EXISTS complaints (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user TEXT,
    category TEXT,
    description TEXT,
    location TEXT,
    urgency TEXT,
    date TEXT
)
''')
conn.commit()

In [13]:
def categorize_complaint(text):
    text = text.lower()
    if any(word in text for word in ["road", "pothole", "traffic", "street"]):
        return "Roads"
    elif any(word in text for word in ["light", "lamp", "streetlight", "bulb"]):
        return "Lights"
    elif any(word in text for word in ["garbage", "waste", "trash", "dustbin", "clean"]):
        return "Waste"
    else:
        return "Other"

def extract_location(text):
    match = re.search(r'near\s+([A-Za-z ]+)|at\s+([A-Za-z ]+)', text)
    if match:
        return match.group(1) or match.group(2)
    return "Unknown"

def determine_urgency(text):
    if any(word in text for word in ["urgent", "immediate", "asap", "danger"]):
        return "High"
    elif any(word in text for word in ["soon", "quickly", "tomorrow"]):
        return "Medium"
    else:
        return "Normal"

In [14]:
def chatbot_reply(user, message):
    category = categorize_complaint(message)
    location = extract_location(message)
    urgency = determine_urgency(message)
    date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Save complaint
    cursor.execute('''
        INSERT INTO complaints (user, category, description, location, urgency, date)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (user, category, message, location, urgency, date))
    conn.commit()

    return f"Thanks {user}! Your {category} issue has been recorded. Location: {location}, Urgency: {urgency}."

In [15]:
# Initialize main window
root = tk.Tk()
root.title("Smart City Complaint Chatbot")
root.geometry("600x500")
root.config(bg="#e8f0fe")

# Add chat display area
chat_window = scrolledtext.ScrolledText(root, wrap=tk.WORD, state='disabled', width=70, height=20, bg="#f8f9fa", fg="#000")
chat_window.pack(padx=10, pady=10)

# Add text entry
entry = tk.Entry(root, width=60, font=('Arial', 12))
entry.pack(side=tk.LEFT, padx=10, pady=5)

user_name = "Citizen"

In [16]:
def send_message():
    user_input = entry.get().strip()
    if not user_input:
        return

    # Display user message
    chat_window.config(state='normal') 
    chat_window.insert(tk.END, f"You: {user_input}\n")
    chat_window.config(state='disabled')
    chat_window.yview(tk.END)

    entry.delete(0, tk.END)

    # Check for special commands
    if user_input.lower() == "show":
        cursor.execute("SELECT * FROM complaints")
        rows = cursor.fetchall()
        chat_window.config(state='normal')
        if not rows:
            chat_window.insert(tk.END, "Bot: No complaints yet.\n")
        else:
            chat_window.insert(tk.END, "Bot: Here are all recorded complaints:\n")
            for r in rows:
                chat_window.insert(tk.END, f"   [#{r[0]}] {r[1]} | {r[2]} | {r[3]} | {r[4]} | {r[5]} | {r[6]}\n")
        chat_window.config(state='disabled')
        chat_window.yview(tk.END)
        return

    if user_input.lower() == "exit":
        root.destroy()
        return

    # Get bot reply
    reply = chatbot_reply(user_name, user_input)
    chat_window.config(state='normal')
    chat_window.insert(tk.END, f"Bot: {reply}\n")
    chat_window.config(state='disabled')
    chat_window.yview(tk.END)

In [17]:
send_button = tk.Button(root, text="Send", command=send_message, bg="#4285F4", fg="white", font=('Arial', 12, 'bold'))
send_button.pack(side=tk.RIGHT, padx=10, pady=5)

root.bind('<Return>', lambda event: send_message())

'1822609035648<lambda>'

In [18]:
chat_window.config(state='normal')
chat_window.insert(tk.END, "🤖 Bot: Hello! I'm your Smart City Complaint Chatbot.\n")
chat_window.insert(tk.END, "You can report problems like:\n")
chat_window.insert(tk.END, " - 'There is garbage near Market Road'\n")
chat_window.insert(tk.END, " - 'Streetlight not working at Park Street'\n")
chat_window.insert(tk.END, "Type 'show' to see all complaints or 'exit' to close.\n\n")

chat_window.config(state='disabled')

root.mainloop()